<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install graphviz

In [39]:
#Importing libraries
import pandas as pd
import numpy as np
from graphviz import Digraph
from matplotlib import pyplot as plt

In [40]:
df = pd.read_csv("Helpdesk.csv")

In [41]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [43]:
df['time:timestamp'] = df['time:timestamp'].str[:10]

In [44]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'],dayfirst= True, format='mixed')

In [45]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [46]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [47]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [48]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [49]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2'],axis=1, inplace=True)

In [53]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [54]:
df=pd.merge(df,stage,on='Activity', how='left')

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   lifecycle:transition  21348 non-null  object        
 1   time:timestamp        21348 non-null  datetime64[ns]
 2   Activity              21348 non-null  object        
 3   Resource              21348 non-null  object        
 4   case:concept:name     21348 non-null  object        
 5   case:variant          21348 non-null  object        
 6   case:variant-index    21348 non-null  int64         
 7   case:creator          21348 non-null  object        
 8   Stage_no              21268 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 1.5+ MB


In [61]:
mv

,lifecycle:transition,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator,Stage_no
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
21343,False,False,False,False,False,False,False,False,False
21344,False,False,False,False,False,False,False,False,False
21345,False,False,False,False,False,False,False,False,False
21346,False,False,False,False,False,False,False,False,False


In [50]:
df.head()

,lifecycle:transition,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,complete,2010-01-13,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,complete,2010-01-29,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,complete,2010-01-29,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,complete,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,complete,2010-02-13,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [18]:
pivot = pd.pivot_table(df,
                       values=['org:resource', 'case:variant'],
                       index='case:concept:name',
                       columns='Activity',
                       aggfunc='count')